In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import os
import pickle
import sklearn
import sys

from sklearn.externals import joblib
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [3]:
%aimport data.load_bibsonomy
%aimport features.preprocess, features.build_features
%aimport helpers.files,helpers.labels

In [57]:
from data.load_bibsonomy import load_to_dataframe_bibtex, load_to_dataframe_bookmark
from helpers.files import get_directory_name_from_hash
from helpers.labels import truncate_labels
from features.preprocess import clean_text_bibtex
from features.build_features import make_bibtex_data, make_bookmark_data

In [5]:
pd.options.display.max_columns = 40

In [30]:
ROOT_TRAIN = "/media/felipe/SAMSUNG/ecml-pkdd-2009/train/2009-01-01_cleaned_post-core-2/"
BIBTEX_TRAIN = ROOT_TRAIN+"bibtex-unix"
BOOKMARK_TRAIN = ROOT_TRAIN+"bookmark-unix"
TAS_TRAIN = ROOT_TRAIN+"tas"

ROOT_TEST = "/media/felipe/SAMSUNG/ecml-pkdd-2009/test/"
BIBTEX_TEST = ROOT_TEST+"2009-07-01_task2/bibtex-unix"
BOOKMARK_TEST = ROOT_TEST+"2009-07-01_task2/bookmark-unix"
TAS_TEST = ROOT_TEST+"2009-07-01_task2_tas_original"

INTERIM_DATA_ROOT = "../data/interim/ecml-pkdd-2009/"
MODELS_ROOT = "../models/"


MAX_NB_WORDS = 20000
LABELS_MIN_DOC_FRACTION = 0.000

In [7]:
if os.path.isfile(INTERIM_DATA_ROOT+"bibtex_docs_df.p"):
    bibtex_docs_df = pickle.load(open(INTERIM_DATA_ROOT+"bibtex_docs_df.p", "rb"))
else:
    bibtex_docs_df = load_to_dataframe_bibtex(BIBTEX_TRAIN,TAS_TRAIN)
    pickle.dump(bibtex_docs_df,open(INTERIM_DATA_ROOT+"bibtex_docs_df.p","wb"))

In [54]:
len(bibtex_docs_df)

22846

# bibtex

In [10]:
data_train = make_bibtex_data(bibtex_docs_df)

In [11]:
data_train[5][:1000]

' Emergent semantics The article discusses ways to let semantics emerge from simple observations from the bottom-up, rather than imposing concepts on the observations top-down, to provide precise query, retrieval, communication or translation for a wide variety of applications. The following areas are examined: image retrieval and databases; media information spaces including the Semantic Web and MPEG frameworks; language games for emergent semantics; and emergent semantics for ontologies kdd  '

In [12]:
tag_sets_train = bibtex_docs_df["tags"].values

all_tags_train = set()

for tag_set in tag_sets_train:
    for tag in tag_set.split(','):
        all_tags_train.add(tag)

In [61]:
len(all_tags_train)

10663

In [15]:
binary_labels_train.shape

(22846, 5815)

### load test data

In [31]:
bibtex_docs_df_test = load_to_dataframe_bibtex(BIBTEX_TEST,TAS_TEST)

data_test = make_bibtex_data(bibtex_docs_df_test)

In [32]:
bibtex_docs_df_test

,content_id,journal,volume,chapter,edition,month,day,booktitle,howPublished,institution,organization,publisher,address,school,series,bibtexKey,url,type,description,annote,note,pages,bKey,number,crossRef,misc,bibtexAbstract,hash0,hash1,hash2,entrytype,title,author,editor,year,tags,contents
0,7024473,Knowl. Acquis.,5,,,June,,,,,,Academic Press Ltd.,"London, UK, UK",,,Gruber93,http://dx.doi.org/10.1006/knac.1993.1008,,,,,199--220,,2,,"posted-at = {2005-09-17 15:18:31}, issn = {104...",,a18748a7f5e0fe51bb1fd562619601d2,232576339f9eecc6915dec6a2ee77150,68f6dc6852686794589bf5e2a9151cd3,article,A translation approach to portable ontology sp...,Thomas R. Gruber,,1993,"masterthesis,ontology,definition",A translation approach to portable ontology s...
1,7028163,,,,,,,Lecture Notes in Computer Science: On the Move...,,,,Springer,,,,christiaens06,http://www.springerlink.com/content/m370107220...,,,,,,,,,"pdf = {christiaens06-metadata.pdf}, lastname =...",In this paper we give a brief overview of diff...,eed329cee527b84b66e271ae2ba86730,f733d993459329ed1ef9f26d303ba0d9,efc1396e845f3db1688dc8ef154d9520,incollection,Metadata Mechanisms: From Ontology to Folksono...,Stijn Christiaens,,2006,"ontology,masterthesis,folksonomy,metadata",Metadata Mechanisms: From Ontology to Folkson...
2,7028464,,,,,,,,,Carnegie Mellon University,,,,Carnegie Mellon University CMU-ISRI-04-105,,Tsvetovat2004DyNetML:-Interc,,,,,,,,CMU-ISRI-04-105,,"date-added = {2006-01-10 14:26:36 -0500}, date...",,2001d15e92f240cbccf20a5980557e63,6e71b40faa9d15ae4e25edb7dafb0375,e5579e07a85d9102213e188658ff5647,techreport,DyNetML: Interchange Format for Rich Social Ne...,Maksim Tsvetovat and Jeff Reminga and Kathleen...,,2004,"socialnetworkanalysis,software",DyNetML: Interchange Format for Rich Social N...
3,7029227,,,,,,,,,,,Harvard Business School Press,Boston,,,Brown:Life,http://books.google.com/books?id=D-WjL_HRbNQC,,,,,,,,,,,b48cd3519b89b50ae759c7e744ff49ae,9a3e9ac501542b592c8e7802f6917601,f31f34c0987c775b49e42cb986c6a3f3,book,The social life of information,John Seely Brown and Paul Duguid,,2002,"toread,knowledgemanagement,learning",The social life of information
4,7034082,,,18,,,,Social Semantic Web,,,,Springer,"Berlin, Heidelberg",,X.media.press,hotho2008social,http://dx.doi.org/10.1007/978-3-540-72216-8_18,,SpringerLink - Buchkapitel,,,363--391,,,,"issn = {1439-3107}, isbn = {978-3-540-72215-1}...",BibSonomy ist ein kooperatives Verschlagwortun...,16b8f07a41097cc4e4fc962eaf809465,79dbca4289cfe913aa7f7eb7e0dccea7,5ccf05a86e7f1a089ae83dd47568e6de,incollection,Social Bookmarking am Beispiel BibSonomy,Andreas Hotho and Robert Jäschke and Dominik B...,Andreas Blumauer and Tassilo Pellegrini,2009,"l3s,myown,fornepomuk,social,bibsonomy,bookmark...",Social Bookmarking am Beispiel BibSonomy BibS...
5,7034180,"Web Semantics: Science, Services and Agents on...",6,,,Feb,,Semantic Web and Web 2.0,,,,Elsevier,New York,,,jaeschke2008discovering,http://www.sciencedirect.com/science/article/B...,,"ScienceDirect - Web Semantics: Science, Servic...",,,38--53,,1,,"issn = {1570-8268}, vgwort = {59}, doi = {10.1...",Social bookmarking tools are rapidly emerging ...,3b7846e6fe5d34080ef2397d8764dd36,cfca594f9dbe30694bfbcdeb40dc4e88,18e8babe208fae2c0342438617b0ec31,article,Discovering Shared Conceptualizations in Folks...,Robert Jäschke and Andreas Hotho and Christoph...,T. Finin and R. Mizoguchi and S. Staab,2008,"l3s,myown,fornepomuk,concept,folksonomy,analys...",Discovering Shared Conceptualizations in Folk...
6,7034196,AI Communications,21,,,,,,,,,IOS Press,Amsterdam,,,jaeschke2008tag,http://dx.doi.org/10.3233/AIC-2008-0438,,,,,231-247,,4,,"issn = {0921-7126}, vgwort = {63}, doi = {10.3...",Collaborative tagging systems allow users to a...,9e9f1e49306d787521a1ca4c336ef787,b2f1aba6829affc85d852ea93a8e39f7,955bcf14f3272ba6eaf3dadbef6c0b10,article,Tag Recommendations in Social Bookmarking Systems,Robert Jäschke and Leandro Marinho and Andreas...,Enrico Giunchiglia,2008,"tag,webzu,myown,2008,recomm

### binarize labels

In [33]:
tag_sets_train = bibtex_docs_df["tags"].values

all_tags_train = set()

for tag_set in tag_sets_train:
    for tag in tag_set.split(','):
        all_tags_train.add(tag)

In [34]:
len(all_tags_train)

5815

In [35]:
min_nb_docs = int(len(bibtex_docs_df)* LABELS_MIN_DOC_FRACTION)
min_nb_docs

0

In [42]:
# train
bibtex_docs_df["tags_split"] = bibtex_docs_df["tags"].map(lambda tagstring: tagstring.split(","))
labels_train = bibtex_docs_df["tags_split"].values
truncated_labels_train = truncate_labels(labels_train,min_nb_docs)

# test
bibtex_docs_df_test["tags_split"] = bibtex_docs_df_test["tags"].map(lambda tagstring: tagstring.split(","))
labels_test = bibtex_docs_df_test["tags_split"].values

all_labels = np.hstack([truncated_labels_train,labels_test])

# fit the binarizer on all data (this is not snooping)
mlb = MultiLabelBinarizer()
mlb.fit(all_labels)

# transform individually train and test
binary_labels_train = mlb.transform(truncated_labels_train)
binary_labels_test = mlb.transform(labels_test)

In [45]:
binary_labels_train.shape,binary_labels_test.shape

((22846, 5831), (347, 5831))

In [ ]:
# use later
# parameters = {
#     'vect__preprocessor': clean_text,
#     "vect__max_features": [MAX_NB_WORDS]
# }

### calculate cv scores

In [46]:
scores = cross_val_score(pipeline_bibtex, data_train, binary_labels_train, cv=3,scoring='f1_micro')

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 11 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 22 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 19 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 17 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 48 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/skl

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 241 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 256 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 261 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 263 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 277 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 302 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packa

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 587 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 607 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 605 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 603 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 613 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 620 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packa

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 892 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 908 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 910 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 911 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 907 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 913 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packa

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1197 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1210 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1214 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1220 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1228 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1250 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site

...

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4977 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4974 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4986 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4993 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4998 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5009 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5175 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5188 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5182 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5190 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5208 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5212 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5362 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5383 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5400 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5401 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5404 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5390 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5567 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5559 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5592 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5577 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5593 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5578 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5736 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5737 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5738 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5756 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5757 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5730 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site

In [47]:
scores.mean()

0.18861752142783553

## now train on the whole thing and test on the actual test set

In [48]:
if os.path.isfile(MODELS_ROOT+'pipeline-bibtex-ovr-linear-svm.p'):
    pipeline_bibtex = joblib.load(MODELS_ROOT+'pipeline-bibtex-ovr-linear-svm.p')
else:
    pipeline_bibtex = Pipeline([
        ('vect', CountVectorizer(preprocessor=clean_text_bibtex, max_features=MAX_NB_WORDS, strip_accents='unicode')),
        ('tfidf', TfidfTransformer()),
        ('clf', OneVsRestClassifier(LinearSVC(),n_jobs=-1)),
    ])
    pipeline_bibtex.fit(data_train,binary_labels_train)
    joblib.dump(pipeline_bibtex, MODELS_ROOT+'pipeline-bibtex-ovr-linear-svm.p')

/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 141 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 185 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 297 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 433 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1214 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1576 is present in all training examples.
  str(classes[c]))
/home/felipe/tf-venv3/lib/python3.5/site-pac

In [50]:
y_pred = pipeline_bibtex.predict(data_test)

In [52]:
y_true = binary_labels_test

In [53]:
f1_score(y_true,y_pred,average='micro')

0.27055393586005833

## bookmark

In [8]:
if os.path.isfile(INTERIM_DATA_ROOT+"bookmark_docs_df.p"):
    bookmark_docs_df = pickle.load(open(INTERIM_DATA_ROOT+"bookmark_docs_df.p", "rb"))
else:
    bookmark_docs_df = load_bookmark_and_tags(BOOKMARK_TRAIN,TAS_TRAIN)
    pickle.dump(bookmark_docs_df,open(INTERIM_DATA_ROOT+"bookmark_docs_df.p","wb"))

In [64]:
bookmark_docs_df

,content_id,url_hash,url,description,extended_description,date,tags,contents
0,8,7edfc1f9560521e83bcf5a5768889c6c,http://jo.irisson.free.fr/bstdatabase/,LaTeX Bibliography Styles Database :: Search,,2005-12-13 08:42:37,"latex,database,style,bibtex",http://jo.irisson.free.fr/bstdatabase/ LaTeX ...
1,11,e636edf2736cfc61897bf21039ffea1b,http://acmqueue.com/modules.php?name=Content&p...,Social Bookmarking in the Enterprise,,2005-12-07 09:08:51,"folksonomy,tagging,enterprise,ibm",http://acmqueue.com/modules.php?name=Content&...
2,12,2f87c060c8ada01d4500e8a27749dee8,http://www.cs.stir.ac.uk/~kjt/software/latex/s...,BibTeX Style Examples,,2005-12-06 13:57:37,"latex,example,bibtex",http://www.cs.stir.ac.uk/~kjt/software/latex/...
3,13,bfb258bc024470f88f8d38c2c4d820ab,http://virtual.cvut.cz:8080/ksmsaWeb/browser/t...,The KSMSA Project - Ontology Browser,,2005-12-02 09:46:10,"browser,sumo,ontology",http://virtual.cvut.cz:8080/ksmsaWeb/browser/...
4,18,0f12bf6a77453df42bd6cf8ccb9ff10a,http://www.cs.utexas.edu/users/mfkb/related.html,KBS/Ontology Projects Worldwide,,2005-11-24 19:09:42,ontology,http://www.cs.utexas.edu/users/mfkb/related.h...
5,21,f7186c61437b071849077a9d74b3a1d1,http://pier.cs.berkeley.edu/,The PIER Project,,2005-11-22 09:25:15,"simulation,simulator,p2p",http://pier.cs.berkeley.edu/ The PIER Project
6,23,1c8096ac2d56e1c521b605aa3b9b151f,http://www.last.fm/,Last.fm,,2005-11-18 11:11:35,"folksonomy,tagging,collaborative,music",http://www.last.fm/ Last.fm
7,29,19cfec376f5b0e4ccddfdc802decdbb9,http://www.alphaworks.ibm.com/tech/uima,alphaWorks : Unstructured Information Manageme...,,2005-11-10 11:37:45,"datamining,machinelearning,framework",http://www.alphaworks.ibm.com/tech/uima alpha...
8,30,1dd1689b0613587b44aa3422b78b6e16,http://www.research.ibm.com/UIMA/,UIMA,,2005-11-10 11:37:36,"datamining,machinelearning,framework",http://www.research.ibm.com/UIMA/ UIMA
9,32,4986c0ef7bf79413faf19f7c9b48eda5,http://www.analytictech.com/ucinet.htm,UCINET 6 Social Network Analysis Software,,2006-09-06 10:34:37,"sna,tool",http://www.analytictech.com/ucinet.htm UCINET...


In [58]:
data_train = make_bookmark_data(bookmark_docs_df)

In [59]:
tag_sets_train = bookmark_docs_df["tags"].values

all_tags_train = set()

for tag_set in tag_sets_train:
    for tag in tag_set.split(','):
        all_tags_train.add(tag)

In [60]:
len(all_tags_train)

10663

In [63]:
min_nb_docs = int(len(bookmark_docs_df)* LABELS_MIN_DOC_FRACTION)
min_nb_docs

0

In [65]:
bookmark_docs_df_test = load_to_dataframe_bookmark(BOOKMARK_TEST,TAS_TEST)

data_test = make_bookmark_data(bookmark_docs_df_test)

In [66]:
tag_sets = bookmark_docs_df["tags"].values

all_tags = set()

for tag_set in tag_sets:
    for tag in tag_set.split(','):
        all_tags.add(tag)

In [67]:
len(all_tags)

10663

In [68]:
min_nb_docs = int(len(bookmark_docs_df)* LABELS_MIN_DOC_FRACTION)
min_nb_docs

0

In [70]:
# train
bookmark_docs_df["tags_split"] = bookmark_docs_df["tags"].map(lambda tagstring: tagstring.split(","))
labels_train = bookmark_docs_df["tags_split"].values
truncated_labels_train = truncate_labels(labels_train,min_nb_docs)

# test
bookmark_docs_df["tags_split"] = bookmark_docs_df["tags"].map(lambda tagstring: tagstring.split(","))
labels_test = bookmark_docs_df["tags_split"].values

all_labels = np.hstack([truncated_labels_train,labels_test])

# fit the binarizer on all data (this is not snooping)
mlb = MultiLabelBinarizer()
mlb.fit(all_labels)

# transform individually train and test
binary_labels_train = mlb.transform(truncated_labels_train)
binary_labels_test = mlb.transform(labels_test)

In [72]:
pipeline_bookmark = Pipeline([
    ('vect', CountVectorizer(max_features=MAX_NB_WORDS, strip_accents='unicode')),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(),n_jobs=-1)),
])

In [ ]:
scores = cross_val_score(pipeline_bookmark, data_train, binary_labels_train, cv=3,scoring='f1_micro')

In [ ]:
#scores.mean()

In [ ]:
if os.path.isfile(MODELS_ROOT+'pipeline-bookmark-ovr-linear-svm.p'):
    pipeline_bookmark = joblib.load(MODELS_ROOT+'pipeline-bookmark-ovr-linear-svm.p')
else:
    pipeline_bookmark.fit(data,binary_label_data)
    joblib.dump(pipeline_bookmark, MODELS_ROOT+'pipeline-bookmark-ovr-linear-svm.p')